# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port = 27017)

In [3]:
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'fruits_db', 'local', 'queries', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
food_db = mongo["uk_food"]

In [5]:
# review the collections in our database
print(food_db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = food_db["establishments"]

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
display_query = establishments.find(query)

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(food_db.establishments.find_one(query))

0
None


In [8]:
import pandas as pd

In [9]:
# Convert the result to a Pandas DataFrame
query_df = pd.DataFrame(display_query)

# Display the number of rows in the DataFrame
print(len(query_df))

# Display the first 10 rows of the DataFrame
query_df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": "4"}}
query2_results = establishments.find(query2)

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query2))

# Display the first document in the results using pprint
pprint(food_db.establishments.find_one(query2))

0
None


In [11]:
# Convert the result to a Pandas DataFrame
query2_df = list(establishments.find(query2))
query2_pd = pd.DataFrame(query2_df)

# Display the number of rows in the DataFrame
print(len(query2_pd))

# Display the first 10 rows of the DataFrame
query2_pd.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
        "RatingValue": {"$eq": "5"}}

sort = [("scores.Hygiene", 1)]

result = list(establishments.find(query).sort(sort).limit(5))

# Print the results
pprint(result)

[]


In [13]:
# Convert result to Pandas DataFrame
pd3 = pd.DataFrame(result)
pd3

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
hygiene_match = {"$match": {"scores.Hygiene": {"$eq": 0}}}

# 2. Groups the matches by Local Authority
groups = {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}

# 3. Sorts the matches from highest to lowest
sort = {"$sort": {"count": -1}}
pipline = [hygiene_match, groups, sort]
results = list(establishments.aggregate(pipline))

# Print the number of documents in the result
pprint(len(results))

# Print the first 10 results
pprint(results[0:10])

0
[]


In [15]:
# Convert the result to a Pandas DataFrame
results_pd = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(len(results_pd))

# Display the first 10 rows of the DataFrame
results_pd.head(10)

0


""
